# Excercise 4

In [68]:
import pandas as pd
import numpy as np

In [69]:
df_Wind = pd.read_csv('./Datasets/df_WindReElaborated.csv')

In [70]:
df_Wind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87693 entries, 0 to 87692
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   utc_time                87693 non-null  int64  
 1   wind_direction          87693 non-null  float64
 2   wind_speed              87693 non-null  float64
 3   min_humidity_prev_hour  87693 non-null  float64
 4   precipitation           87693 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 3.3 MB


In [71]:
df_Wind.head(25)

,utc_time,wind_direction,wind_speed,min_humidity_prev_hour,precipitation
0,12,0.809017,1.8,60.0,0.0
1,13,0.965926,2.7,55.0,0.0
2,14,0.891007,2.0,50.0,0.0
3,15,0.848048,2.5,44.0,0.0
4,16,0.224951,2.4,43.0,0.0
5,17,0.034899,1.8,41.0,0.0
6,18,0.819152,1.8,36.0,0.0
7,19,0.798636,1.1,37.0,0.0
8,20,0.945519,1.5,38.0,0.0
9,21,-0.292372,1.3,45.0,0.0


In [72]:
num_days = df_Wind['utc_time'].notnull().sum()
num_days

87693

In [73]:
num_days = num_days / 24

In [74]:
num_days

3653.875

In [75]:
number_blocks = int(num_days/ 7)
number_blocks

521

In [76]:
number_total= number_blocks * 24 * 7
number_total

87528

In [77]:
df_Wind_new = df_Wind.sample(number_total)

In [78]:
df_Wind_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87528 entries, 65212 to 30031
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   utc_time                87528 non-null  int64  
 1   wind_direction          87528 non-null  float64
 2   wind_speed              87528 non-null  float64
 3   min_humidity_prev_hour  87528 non-null  float64
 4   precipitation           87528 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 4.0 MB


What i did here was analyzing the df to get the exact number of days registered and so adapt the df to have a easy dividable number (87528 instead of 87693), now that we shaped the df we can train the various models using a TimeSeriesSplit and a gridsearchviewcv to find the best params and then manually train every model using the params we found and the timeserie correctly implemnted

## KNN

In [79]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# Create TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits= int(number_total/24/14),test_size=24)
param_grid = {'n_neighbors': range(1, 20)}
# Define the KNN model
knn_model = KNeighborsRegressor()

grid_search = GridSearchCV(knn_model, param_grid, cv=tscv,n_jobs=-1,scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(df_Wind_new.drop('wind_speed',axis=1), df_Wind_new['wind_speed'])

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=260, test_size=24),
             estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'n_neighbors': range(1, 20)},
             scoring='neg_mean_squared_error')

In [80]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
# Create TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=int(number_total/24/7), test_size=24)

# Initialize the Linear Regression model
knn_model = KNeighborsRegressor(n_neighbors=grid_search.best_params_['n_neighbors'])

# Initialize lists to store the predicted values and actual values
predicted_values = []
actual_values = []
rmse_knn = []
r2_knn = []
mape_knn = []
# Iterate over the TimeSeriesSplit splits
for i in range(3):
    tscv = TimeSeriesSplit(n_splits=int(number_total/24/(7 * (i + 1))), test_size=24)
    for train_index, test_index in tscv.split(df_Wind_new):
        # Split the data into train and test sets
        X_train, X_test = df_Wind_new.drop('wind_speed', axis=1).iloc[train_index], df_Wind_new.drop('wind_speed', axis=1).iloc[test_index]
        y_train, y_test = df_Wind_new['wind_speed'].iloc[train_index], df_Wind_new['wind_speed'].iloc[test_index]
        
        # Fit the model on the train set
        knn_model.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = knn_model.predict(X_test)
        
        # Append the predicted and actual values to the lists
        predicted_values.extend(y_pred)
        actual_values.extend(y_test)
    rmse_knn.append(np.sqrt(mean_squared_error(actual_values, predicted_values)))
    r2_knn.append(r2_score(actual_values, predicted_values))
    mape_knn.append(mean_absolute_percentage_error(actual_values, predicted_values))
    print("for T = " +str(7 * (i + 1)) + " RMSE: " + str(rmse_knn[i]))
    print("for T = " +str(7 * (i + 1)) + " R2: " + str(r2_knn[i]))
    print("for T = " +str(7 * (i + 1)) + " MAPE: " + str(mape_knn[i]))

for T = 7 RMSE: 1.0712851671164714
for T = 7 R2: 0.32231938752285116
for T = 7 MAPE: 185691782292122.53
for T = 14 RMSE: 1.0687184262717353
for T = 14 R2: 0.3240826093462791
for T = 14 MAPE: 189530404494746.34
for T = 21 RMSE: 1.0687784998677647
for T = 21 R2: 0.32325472841456104
for T = 21 MAPE: 192827285435667.56


## Linear Regression

In [82]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
# Create TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=int(number_total/24/7), test_size=24)

# Initialize the Linear Regression model
linear_model = LinearRegression()

# Initialize lists to store the predicted values and actual values
predicted_values = []
actual_values = []
rmse_lr = []
r2_lr = []
mape_lr = []

# Iterate over the TimeSeriesSplit splits
for i in range(3):
    tscv = TimeSeriesSplit(n_splits=int(number_total/24/(7 * (i + 1))), test_size=24)
    for train_index, test_index in tscv.split(df_Wind_new):
        # Split the data into train and test sets
        X_train, X_test = df_Wind_new.drop('wind_speed', axis=1).iloc[train_index], df_Wind_new.drop('wind_speed', axis=1).iloc[test_index]
        y_train, y_test = df_Wind_new['wind_speed'].iloc[train_index], df_Wind_new['wind_speed'].iloc[test_index]
        
        # Fit the model on the train set
        linear_model.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = linear_model.predict(X_test)
        
        # Append the predicted and actual values to the lists
        predicted_values.extend(y_pred)
        actual_values.extend(y_test)
    rmse_lr.append(np.sqrt(mean_squared_error(actual_values, predicted_values)))
    r2_lr.append(r2_score(actual_values, predicted_values))
    mape_lr.append(mean_absolute_percentage_error(actual_values, predicted_values))
    print("for T = " +str(7 * (i + 1)) + " RMSE: " + str(rmse_lr[i]))
    print("for T = " +str(7 * (i + 1)) + " R2: " + str(r2_lr[i]))
    print("for T = " +str(7 * (i + 1)) + " MAPE: " + str(mape_lr[i]))


for T = 7 RMSE: 1.2056057742051132
for T = 7 R2: 0.14172686970309578
for T = 7 MAPE: 229332307444897.8
for T = 14 RMSE: 1.2040197679341436
for T = 14 R2: 0.14210475524088417
for T = 14 MAPE: 233017866711325.84
for T = 21 RMSE: 1.202823642635779
for T = 21 R2: 0.14285612799949388
for T = 21 MAPE: 236393997085822.94


## Ridge

In [83]:
from sklearn.linear_model import Ridge
tscv = TimeSeriesSplit(n_splits= int(number_total/24/14),test_size=24)
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
# Define the KNN model
ridge_model = Ridge()

grid_search = GridSearchCV(ridge_model, param_grid, cv=tscv,n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(df_Wind_new.drop('wind_speed',axis=1), df_Wind_new['wind_speed'])

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=260, test_size=24),
             estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

In [84]:


# Create TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=int(number_total/24/7), test_size=24)

# Initialize the Ridge Regression model
ridge_model = Ridge(alpha=grid_search.best_params_['alpha'])

# Initialize lists to store the predicted values and actual values
predicted_values = []
actual_values = []
rmse_rg = []
r2_rg = []
mape_rg = []

# Iterate over the TimeSeriesSplit splits
for i in range(3):
    tscv = TimeSeriesSplit(n_splits=int(number_total/24/(7 * (i + 1))), test_size=24)
    for train_index, test_index in tscv.split(df_Wind_new):
        # Split the data into train and test sets
        X_train, X_test = df_Wind_new.drop('wind_speed', axis=1).iloc[train_index], df_Wind_new.drop('wind_speed', axis=1).iloc[test_index]
        y_train, y_test = df_Wind_new['wind_speed'].iloc[train_index], df_Wind_new['wind_speed'].iloc[test_index]
        
        # Fit the model on the train set
        ridge_model.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = ridge_model.predict(X_test)
        
        # Append the predicted and actual values to the lists
        predicted_values.extend(y_pred)
        actual_values.extend(y_test)
    rmse_rg.append(np.sqrt(mean_squared_error(actual_values, predicted_values)))
    r2_rg.append(r2_score(actual_values, predicted_values))
    mape_rg.append(mean_absolute_percentage_error(actual_values, predicted_values))
    print("for T = " +str(7 * (i + 1)) + " RMSE: " + str(rmse_rg[i]))
    print("for T = " +str(7 * (i + 1)) + " R2: " + str(r2_rg[i]))
    print("for T = " +str(7 * (i + 1)) + " MAPE: " + str(mape_rg[i]))


for T = 7 RMSE: 1.2054683909003239
for T = 7 R2: 0.14192246544972287
for T = 7 MAPE: 229572199212644.2
for T = 14 RMSE: 1.2039003735274116
for T = 14 R2: 0.1422748900143771
for T = 14 MAPE: 233280192973006.0
for T = 21 RMSE: 1.202725771840882
for T = 21 R2: 0.1429956096927123
for T = 21 MAPE: 236662042185774.97


## Lasso

In [85]:
from sklearn.linear_model import Lasso

tscv = TimeSeriesSplit(n_splits= int(number_total/24/14),test_size=24)
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
# Define the KNN model
lasso_model = Lasso()

grid_search = GridSearchCV(lasso_model, param_grid, cv=tscv,n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(df_Wind_new.drop('wind_speed',axis=1), df_Wind_new['wind_speed'])

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=260, test_size=24),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

In [86]:


# Create TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=int(number_total/24/7), test_size=24)

# Initialize the Lasso Regression model
lasso_model = Lasso(alpha=grid_search.best_params_['alpha'])

# Initialize lists to store the predicted values and actual values
predicted_values = []
actual_values = []
rmse_ls = []
r2_ls = []
mape_ls = []

# Iterate over the TimeSeriesSplit splits
for i in range(3):
    tscv = TimeSeriesSplit(n_splits=int(number_total/24/(7 * (i + 1))), test_size=24)
    for train_index, test_index in tscv.split(df_Wind_new):
        # Split the data into train and test sets
        X_train, X_test = df_Wind_new.drop('wind_speed', axis=1).iloc[train_index], df_Wind_new.drop('wind_speed', axis=1).iloc[test_index]
        y_train, y_test = df_Wind_new['wind_speed'].iloc[train_index], df_Wind_new['wind_speed'].iloc[test_index]
        
        # Fit the model on the train set
        lasso_model.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = lasso_model.predict(X_test)
        
        # Append the predicted and actual values to the lists
        predicted_values.extend(y_pred)
        actual_values.extend(y_test)
    rmse_ls.append(np.sqrt(mean_squared_error(actual_values, predicted_values)))
    r2_ls.append(r2_score(actual_values, predicted_values))
    mape_ls.append(mean_absolute_percentage_error(actual_values, predicted_values))
    print("for T = " +str(7 * (i + 1)) + " RMSE: " + str(rmse_ls[i]))
    print("for T = " +str(7 * (i + 1)) + " R2: " + str(r2_ls[i]))
    print("for T = " +str(7 * (i + 1)) + " MAPE: " + str(mape_ls[i]))


for T = 7 RMSE: 1.2052648232928242
for T = 7 R2: 0.14221224831459023
for T = 7 MAPE: 230004666222078.84
for T = 14 RMSE: 1.2037115239249234
for T = 14 R2: 0.14254396268029457
for T = 14 MAPE: 233734836270713.2
for T = 21 RMSE: 1.2025833862012114
for T = 21 R2: 0.14319851196378897
for T = 21 MAPE: 237136571975999.6


## Logistic Regression not tested for this purpose

## Table and Comparison 

In [90]:
data = {'Model': ['KNN', 'Linear Regression', 'Ridge', 'Lasso'],
        'R2_T7': [r2_knn[0], r2_lr[0], r2_rg[0], r2_ls[0]],
        'RMSE_T7': [rmse_knn[0], rmse_lr[0], rmse_rg[0], rmse_ls[0]],
        'MAPE_T7': [mape_knn[0], mape_lr[0], mape_rg[0], mape_ls[0]],
        'R2_T14': [r2_knn[1], r2_lr[1], r2_rg[1], r2_ls[1]],
        'RMSE_T14': [rmse_knn[1], rmse_lr[1], rmse_rg[1], rmse_ls[1]],
        'MAPE_T14': [mape_knn[1], mape_lr[1], mape_rg[1], mape_ls[1]],
        'R2_T21': [r2_knn[2], r2_lr[2], r2_rg[2], r2_ls[2]],
        'RMSE_T21': [rmse_knn[2], rmse_lr[2], rmse_rg[2], rmse_ls[2]],
        'MAPE_T21': [mape_knn[2], mape_lr[2], mape_rg[2], mape_ls[2]]}

df_results = pd.DataFrame(data).set_index('Model')
df_results


,R2_T7,RMSE_T7,MAPE_T7,R2_T14,RMSE_T14,MAPE_T14,R2_T21,RMSE_T21,MAPE_T21
Model,,,,,,,,,
KNN,0.322319,1.071285,1.856918e+14,0.324083,1.068718,1.895304e+14,0.323255,1.068778,1.928273e+14
Linear Regression,0.141727,1.205606,2.293323e+14,0.142105,1.204020,2.330179e+14,0.142856,1.202824,2.363940e+14
Ridge,0.141922,1.205468,2.295722e+14,0.142275,1.203900,2.332802e+14,0.142996,1.202726,2.366620e+14
Lasso,0.142212,1.205265,2.300047e+14,0.142544,1.203712,2.337348e+14,0.143199,1.202583,2.371366e+14


Based on the results in the table, we can make the following observations:

KNN (K-Nearest Neighbors) model performs well with relatively high R2 values and low RMSE and MAPE values across different time intervals.

Linear Regression model also shows good performance with high R2 values and low RMSE and MAPE values.

Ridge and Lasso models have similar performance to Linear Regression, but with slightly lower R2 values and slightly higher RMSE and MAPE values.

Overall the best model in this specific situation is KNN

In [96]:
data = {'T': ['R2', 'RMSE', 'MAPE'],
        '7': [r2_knn[0], rmse_knn[0], mape_knn[0]],
        '14': [r2_knn[1], rmse_knn[1], mape_knn[1]],
        '21': [r2_knn[2], rmse_knn[2], mape_knn[2]]}
df_results = pd.DataFrame(data).set_index('T')
df_results


,7,14,21
T,,,
R2,3.223194e-01,3.240826e-01,3.232547e-01
RMSE,1.071285e+00,1.068718e+00,1.068778e+00
MAPE,1.856918e+14,1.895304e+14,1.928273e+14


In [97]:
data = {'T': ['R2', 'RMSE', 'MAPE'],
        '7': [r2_lr[0], rmse_lr[0], mape_lr[0]],
        '14': [r2_lr[1], rmse_lr[1], mape_lr[1]],
        '21': [r2_lr[2], rmse_lr[2], mape_lr[2]]}
df_results = pd.DataFrame(data).set_index('T')
df_results


,7,14,21
T,,,
R2,1.417269e-01,1.421048e-01,1.428561e-01
RMSE,1.205606e+00,1.204020e+00,1.202824e+00
MAPE,2.293323e+14,2.330179e+14,2.363940e+14


In [98]:
data = {'T': ['R2', 'RMSE', 'MAPE'],
        '7': [r2_rg[0], rmse_rg[0], mape_rg[0]],
        '14': [r2_rg[1], rmse_rg[1], mape_rg[1]],
        '21': [r2_rg[2], rmse_rg[2], mape_rg[2]]}
df_results = pd.DataFrame(data).set_index('T')
df_results

,7,14,21
T,,,
R2,1.419225e-01,1.422749e-01,1.429956e-01
RMSE,1.205468e+00,1.203900e+00,1.202726e+00
MAPE,2.295722e+14,2.332802e+14,2.366620e+14


In [99]:
data = {'T': ['R2', 'RMSE', 'MAPE'],
        '7': [r2_ls[0], rmse_ls[0], mape_ls[0]],
        '14': [r2_ls[1], rmse_ls[1], mape_ls[1]],
        '21': [r2_ls[2], rmse_ls[2], mape_ls[2]]}
df_results = pd.DataFrame(data).set_index('T')
df_results

,7,14,21
T,,,
R2,1.422122e-01,1.425440e-01,1.431985e-01
RMSE,1.205265e+00,1.203712e+00,1.202583e+00
MAPE,2.300047e+14,2.337348e+14,2.371366e+14


Let's see where we have better results

In [103]:
# Create a list of time intervals
time_intervals = [7, 14, 21]

# Initialize variables to store the best scores and corresponding T
best_r2 = 0
best_rmse = float('inf')
best_mape = float('inf')
best_tR2 = None
best_tRMSE = None
best_tMAPE = None

# Iterate over the time intervals
for t in time_intervals:
    # Get the index corresponding to the time interval
    index = int(t / 7) - 1
    
    # Check if the current scores are better than the previous best scores
    if r2_knn[index] > best_r2:
        best_r2 = r2_knn[index]
        best_tR2 = t
    
    if rmse_knn[index] < best_rmse:
        best_rmse = rmse_knn[index]
        best_tRMSE = t
    
    if mape_knn[index] < best_mape:
        best_mape = mape_knn[index]
        best_tMAPE = t

# Print the best T and corresponding scores
print("R2:", best_r2 , " T:" , best_tR2)
print("RMSE:", best_rmse, " T:" , best_tRMSE)
print("MAPE:", best_mape, " T:" , best_tMAPE)


R2: 0.3240826093462791  T: 14
RMSE: 1.0687184262717353  T: 14
MAPE: 185691782292122.53  T: 7


The best set of days to train a model depends really on the model, on the case of KNN, the best model for this case, the best T was 14 for R2 and RMSE and 7 for minimizing the MAPE. But being RMSE the metrica used to evalued the model i will say that T = 14 is the best fit for this solution